In [ ]:
# pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

from collections import Counter

In [2]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_info.csv')
eat_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_review.csv')


In [5]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,5000,30,4.5,37.529946,127.073843,20,# 뚝방길 크림티 쉬는 날 방문짧은 휴가를 여유롭고 사랑스럽게 만들어준 곳. 뚝방길...
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,5000,5,4.2,37.557517,127.081245,61,날이 많이 더워져서 젤라또 먹기 좋은 날씨 같다. 어린이 대공원에서 가깝긴 하지만 ...
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,5000,11,4.2,37.542645,127.070165,3,"커피와 술이 다 있는 곳을 찾다가 들어간 곳-!! 와인이 있길래 시켜서 먹었는데, ..."
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,5000,7,4.2,37.543360,127.070580,99,최가커피 본점을 방문한적이 있었는데 그때 좀 가게가 낡았던....기억이 있어서 분점...
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,5000,34,4.1,37.540869,127.066014,7,꼭 한번 와보고 싶었다. 여기서 커피와 독서를 즐겨보고 싶었다. 결과적으로 나는 대...
...,...,...,...,...,...,...,...,...,...,...
146,메종드쁘띠푸르,서울특별시 광진구 아차산로 200,카페 / 디저트,5000,3,NaN,37.540869,127.066014,38,빵 종류도 많고 빵은 괜찮아요. 벋 테이블 세 개만 내부에 있고 다 외부에요. 겨울...
147,더파빌리온,서울특별시 광진구 워커힐로 177 그랜드 워커힐 본관 1F,카페 / 디저트,15000,13,3.7,37.555213,127.110890,18,피자힐에서 그렇게 거나하게 먹어놓고 디저트가 들어갈 배가 아직 남아있다는게 증맬 놀...
148,줄리앤몰리,서울특별시 광진구 뚝섬로59길 67 1F,카페 / 디저트,15000,4,NaN,37.534731,127.082997,91,"선물하기좋은 귀염뽀짝마카롱,예약없이도 구매가능한 당일레터링케이크,매일 갈수밖에 없는..."
149,케이크를부탁해,서울특별시 광진구 능동로 137,카페 / 디저트,15000,5,NaN,37.543065,127.071800,125,"가격이 워낙 저렴하다보니 특별히 엄청 맛있는 곳은 아니지만, 부담스럽지 않은 가격에..."


In [6]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,스타벅스,양양,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",3,63,416
1,카페샛길,달리자크롱,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",5,110,252
2,투썸플레이스,muk_so.o,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,3,132,148
3,투썸플레이스,뒤빈,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,3,132,266
4,카페구들,심아호,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,5,103,406
...,...,...,...,...,...,...
1194,스타벅스,뽐뭉,층이 여러개인 매장이지만 매장내부에서 층간 연결이 되어있지않아 주문후 다른층으로 이...,0,63,367
1195,탭플레이,수연,생딸기 모히또 맛있네여! 애플민트를 직접 빻아서 만든다고 되어있어요. 모히또 전문점...,5,129,390
1196,투썸플레이스,먹방꼬북,쾌적하고 좋아요,3,132,317
1197,카페모든날,슈빙,친구 강아지 데려갈 수 있는 카페 찾아서 갔던 곳.시그니처 아인슈페너 마셨는데 많이...,3,109,394


In [7]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

  0%|          | 0/600 [00:00<?, ?it/s]

In [8]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [9]:
eat_info.head()

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,5000,30,4.5,37.529946,127.073843,20,# 뚝방길 크림티 쉬는 날 방문짧은 휴가를 여유롭고 사랑스럽게 만들어준 곳. 뚝방길...,
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,5000,5,4.2,37.557517,127.081245,61,날이 많이 더워져서 젤라또 먹기 좋은 날씨 같다. 어린이 대공원에서 가깝긴 하지만 ...,
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,5000,11,4.2,37.542645,127.070165,3,"커피와 술이 다 있는 곳을 찾다가 들어간 곳-!! 와인이 있길래 시켜서 먹었는데, ...",
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,5000,7,4.2,37.543360,127.070580,99,최가커피 본점을 방문한적이 있었는데 그때 좀 가게가 낡았던....기억이 있어서 분점...,
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,5000,34,4.1,37.540869,127.066014,7,꼭 한번 와보고 싶었다. 여기서 커피와 독서를 즐겨보고 싶었다. 결과적으로 나는 대...,


In [10]:
eat_review_by_user.head()

,reviews,tags
416,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",
252,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",
148,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,
266,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,
406,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,


In [11]:
# with open('../data/stop_words.txt','r',-1,'utf-8') as f:
#     stop_words = set(f.read().split('/'))

In [12]:

stopwords = ['맛','더','것','곳','가격','때','밥','진짜','정도','건대','집','여기','음식','좀','처음','수',
             '생각','사람','방문','주문','정말','친구','바로','다른','가게','바','이','조금','또',
             '내','로우','거','하나','메뉴','요리','그','점','가지','별로','식당','한번','추천','삼선','보고','그냥','꽤','보통','나','입','제','직원',
             '다시','말','왜','그릇','무게','꼭','다음','개','느낌','아주','자리','거리','때문','리뷰',
             '중','도','웨이','팅','일단','사진','저','계속','가장','제일','걸','듯','안','건',
             '명','간이','이유','역시','편이','추가','편','재','먹기','거의','시간','기도','테이블','분','마',
             '담','날','근처','손','땐','못','완전','애정','항상','첫','제대로','입맛','이번','먹음',
             '궁합','최고','원래','가면','예전','뭔가','평점','만','한국','다만','기대','배','막','오위','시작',
             '종류','후','냄새','중간','해먹','번','뒤','배달',
             '그램','점점','갑자기','감','살짝','물','콤','색',
             '간','사실','강추','최애','자체','이전','카','골목','음식점','상태','금방','위생','용서','조리','질','요','그때',
             '널','리지','약간','앞','시로','마루','무난','기본','취향','깨','양','줄','수도','개인','매장','식사','두',
             '생','산다','사을','자극','다소','둘','전체','끼','서비스','이이','맛집','아부','얼','비주','오픈','저녁','으;',
             '마끼','대기','운','재료','퀄리티','점심','소스','포장','내부','할머니','토마','식감','동네','일','혼자','요즘','어마','가득','해',
             '저희','볼','인기','뭐','말씀','평일','세','도착','사이즈','일반','밖','간장','후회','인분','손님','달','자주','함',
             '굳이','모든','매우','거기','인생','학교','크기','분위기','절대','주변','무조건','인정','푹','식','옆','주차','위치','존맛','스타일',
             '이모','무엇','밤','새벽','길','기준','알','자꾸','전형','무척','차','베이스','불어','임','위','음','이제','얘기','이해','비','기분',
             '지금','덜','답','북적','신장','개업','끝','방송','위해','법','특유','모습','비교','부분','웨이팅','옥','은','동이','팀','베','부',
             '하의','의사','케이','쿠라','인테리어','마당','대비','인','워낙','쿠니','반','데','마음','개조','미리','단점','하이','','','','',
             '양념','가성','구의역','술집','양도','세트','직접','시징','가족','체인점','안나',
             '이집','저집','적','시','물이','모두','갓','글','수가','집의','합리','부집','준','해도','갈이'
             ]

In [16]:
# pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 7.0 MB/s 


In [17]:
import konlpy
Okt = konlpy.tag.Okt()

def get_tags_to_txt(text):
    Okt_morphs = Okt.pos(str(text)) 
    

    Noun_words = []
    for word, pos in Okt_morphs:
        if pos == 'Noun':
            Noun_words.append(word)
    
    unique_Noun_words = set(Noun_words)
    for word in unique_Noun_words:
        if word in stopwords:  #stopwords
            while word in Noun_words: Noun_words.remove(word)  # 최종결과 : Noun_word
    
    if type(text) != str:
        return ''
    tags = Noun_words
    
    
    tags = ','.join(tags)
    
    return tags

In [18]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    print(get_tags_to_txt(eat_info.loc[i,'reviews']))
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])
    print(get_tags_to_txt(eat_review_by_user.loc[i,'reviews']))

  0%|          | 0/151 [00:00<?, ?it/s]

뚝방길,크림티,휴가,여유,사랑,뚝방길,크림티,플레인,스콘,무화과,스콘,클로티드,크림,딸기,스콘,목,뻑뻑,스콘,클로티드,크림,딸기,스콘,조합,티,웨딩,페리,향,찻잔,티팟,군더더기,평소,홍차,요런,찻잔,공간,홍차,스콘,여유,순간,왼쪽,천상여자,오른쪽,털털,와일드,중성,여자,가운데,바늘,왼쪽,화,악,협소해,간격,리가,점도,잔,가기,뚝섬,안내,센터,주차장,이용,단,단,애프터눈티,애프터눈티,샌드위치,스콘,디저트,구성,시도,명성,게다가,차도,공간,예약,게,애프터눈티,이용,나름,꿀팁,중국,이면,버터,리,영어,이름,홍차,중국,쪽,아이스,찻주전자,찻잔,스콘,움,과자류,디저트,류,달라,하니,타임,일찍,간다,먼저,입장,선착순,시스템,게,반성,프로필사진,뚝방길,에프,터눈티,에프,터눈티,예약,스콘,티,스콘,등,디저트,다기,실내,나중,에프,터눈티,예약,사장,진심,구성,디저트,샌듸치,휘,낭시,스콘,케익,빠,짐,구입,판매,티,대해,공수,사장,영업,여러,홍차,디저트,다수,다가,장소,접근,크림티,밋업,홀릭,뚝방길,오래오래,부터,인스타,기회,결론,이야기,찻집,호불호,갈릴,우선,티,사장,사랑,세계,각국,명차,여러,티,브랜드,블랜디드,티,무시,구비,디카,페인,티,완비,세심,스트레이트,티,선호,밀크,티,바틀,선물,실제,향,향,일품,티,다기,찻잔,티팟,영국,시골,실내,빈티,색감,다기,마치,유럽,행온,착각,잎차,우려,방식,음료,제한,농도,온도,우려,진,티팟,크림티,홍차,구성,식후,티타임,스콘,촉촉,스콘,클로티드,크림,딸기,잼,조합,무적,채광,실내,사이,간격,소리,인구,밀도,온도,기억,티,하우스,앤티크,유럽,홍차,러버,라면,쯤,뚝방길,홍차,홍차,부문,최근,커피,디저트,류,관심,인스타,정보,수집,및,약용,사용,홍차,관심,덕분,평생,밋업,홀릭,덕,먼저,풍경,프랑스,소도시,찻집,게다가,잔뜩,전시,디저트,류,디저트,군침,돌,직,전시,메뉴판,속,상상,초월,디저트,천국,짐작,세계,홍차,스리랑카,장미,향,멘솔,향,향,구분,홍차,홍차,향,떫음,홍차,홍차,향,일시,삶,단계,업그레이드,스콘,너,목,스콘,찻집,디저트,차도,

  0%|          | 0/600 [00:00<?, ?it/s]

오늘,커피,케익,공부,사람과,대화,혼재,노트북,가능,대략,석,대략,퇴근,이후,나머지,널널,주말,오후,쯤,아메리카노,라떼,얼그레이,케이크,포토,스팟,의자,공부,수다,떨기,가기
카페,샛길,가도,내용,글쓰기,디저트,사장,짱,장사,바람,빈티,소품,성,샛길,노래,티라미수,케이크,젠,티라미수,바스크,치즈케이크,케이크,디저트,치즈,치즈케이크,빵,폭,신하,크림,치즈,바닐라,라떼,헤이즐넛,라떼,우유,커피,아메리카노,단,음료,카페,예정,초록,식물,카페,꼬메노,꼬메노,사이,눈,카페,만석,카페,가운데,칸막이,공간,양쪽,출입문,쪽,카페,특징,카페,곳곳,초록,식물,식물,커서,우림,초록,식물,맘,아메리카노,모히또,카페,운영,모히또,한잔,안주,견과류,모히또,술,마시기,라임,향,화장실,사장,서빙,굿,망고,망고,망고,테이크아웃,전문점,잠깐,음료,리터,망고,장난,냉동,망고,통째,망고,아이스크림,또한,생크림,상큼,망고,아쉬움,대신,이서,두고두고,리터,망고,각각,다짐,베스트,식후땡,망고,부담,달콤,망고,그대로,식후,부담,컵홀더,망고,지로,움,망고,빙수,도쿄,빙수,빙수,소품,빙수,컨셉,조심,쟁반,애플,망고,빙수,얼음,빙수,얼음,얼음,연유,사이사이,듬뿍,우유,얼음,빙수,달달,초콜릿,엠엔엠즈,초콜릿,애플,망고,퓨레,냉동,조각,조각,아래,합,애플,망고,시럽,듬뿍,시럽,차지,빙수,빙,토마토,빙수,녹차,빙수,애플,망고,토마토,빙수,화양동,카페,주소,이름,카페,지하,발견,유레카,뿌듯,카페,소리,피크타임,런가,목소리,크게,혼,수제,딸기,라떼,수제,요거트,케이크,딸기,딸기,파티,딸기,라떼,딸기,청,우유,얼음,음료,요거트,케이크,상큼,달달,하니,질리,쉬폰케이크,폭신,과일,디저트,케이크,유일,무,그릭,요거트,왕,맨,호점,장사,도로,변,감각,그릭,요거트,블루,오션,그릭,요거트,저번,그릭,요거트,정성,토핑,골,난,이름,매력,차별,난,상큼,게,과일,위주,참고,비스켓,다이어트,피,꿀,바닥,꿀,요거트,난,달라,업,그릭,요거트,덩이,차고,찍기,하겐다즈,아이스크림,와플,세종대,카페,학생,과제,멀티탭,센스,공부,문제,아이스크림,와

In [19]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,5000,30,4.5,37.529946,127.073843,20,# 뚝방길 크림티 쉬는 날 방문짧은 휴가를 여유롭고 사랑스럽게 만들어준 곳. 뚝방길...,"뚝방길,크림티,휴가,여유,사랑,뚝방길,크림티,플레인,스콘,무화과,스콘,클로티드,크림..."
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,5000,5,4.2,37.557517,127.081245,61,날이 많이 더워져서 젤라또 먹기 좋은 날씨 같다. 어린이 대공원에서 가깝긴 하지만 ...,"젤라또,날씨,어린이,공원,쿠폰,단골,매일,인스타,공지,사장,상큼,과일,상상,고민,리..."
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,5000,11,4.2,37.542645,127.070165,3,"커피와 술이 다 있는 곳을 찾다가 들어간 곳-!! 와인이 있길래 시켜서 먹었는데, ...","커피,술,와인,축하,병,헨켈,두번째,빌라,를,술,술,포함,두번째,케잌,딸,미집,국문..."
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,5000,7,4.2,37.543360,127.070580,99,최가커피 본점을 방문한적이 있었는데 그때 좀 가게가 낡았던....기억이 있어서 분점...,"최,커피,본점,한적,기억,분점,지하,공간,디저트,고민,보관,유리,대신,아포,카토,비..."
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,5000,34,4.1,37.540869,127.066014,7,꼭 한번 와보고 싶었다. 여기서 커피와 독서를 즐겨보고 싶었다. 결과적으로 나는 대...,"커피,독서,결과,만족,서점,독립,출판사,책,디피,책,책,커먼그라운드,층,주차장,마련..."
...,...,...,...,...,...,...,...,...,...,...,...
146,메종드쁘띠푸르,서울특별시 광진구 아차산로 200,카페 / 디저트,5000,3,NaN,37.540869,127.066014,38,빵 종류도 많고 빵은 괜찮아요. 벋 테이블 세 개만 내부에 있고 다 외부에요. 겨울...,"빵,빵,외부,겨울,케이크,퐁당,쇼콜라,파운드,케이크,류,샐러드,커피,커먼그라운드,규..."
147,더파빌리온,서울특별시 광진구 워커힐로 177 그랜드 워커힐 본관 1F,카페 / 디저트,15000,13,3.7,37.555213,127.110890,18,피자힐에서 그렇게 거나하게 먹어놓고 디저트가 들어갈 배가 아직 남아있다는게 증맬 놀...,"피자,힐,거나,디저트,증,빙수,빙수,팥,누구,떡,연유,달라,주시,만큼,마카롱,요건,..."
148,줄리앤몰리,서울특별시 광진구 뚝섬로59길 67 1F,카페 / 디저트,15000,4,NaN,37.534731,127.082997,91,"선물하기좋은 귀염뽀짝마카롱,예약없이도 구매가능한 당일레터링케이크,매일 갈수밖에 없는...","선물,귀염,뽀짝,마카롱,예약,구매,당일,레터,링,케이크,매일,갈수,스페셜,디저트,광..."
149,케이크를부탁해,서울특별시 광진구 능동로 137,카페 / 디저트,15000,5,NaN,37.543065,127.071800,125,"가격이 워낙 저렴하다보니 특별히 엄청 맛있는 곳은 아니지만, 부담스럽지 않은 가격에...","부담,홀,케이크,케이크,생일,선물,다,생일,케이크,바나나,초코,케이크,이서,크림,자..."


In [32]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_info.csv')

In [ ]:
# eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_info.csv')

In [33]:
eat_review_by_user

,reviews,tags
416,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...","오늘,커피,케익,공부,사람과,대화,혼재,노트북,가능,대략,석,대략,퇴근,이후,나머지..."
252,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...","카페,샛길,가도,내용,글쓰기,디저트,사장,짱,장사,바람,빈티,소품,성,샛길,노래,티..."
148,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,"스트로베리,프라,페,딸기,프라페,요거트,음료,바닐라,아이스크림,바닐라,아이스크림,젤..."
266,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,"잠깐,이대,투썸,신규,공간,노트북,대학가,음료,투썸,뱅쇼,따땃,프랜차이즈,뱅쇼,경우..."
406,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,"사장,따숩,아메리카노,구들,커피,오구오구,커피,황달,핀,구들,커피,처,커피,아인슈패..."
429,맛있어요.,
592,건대입구역 바로 옆에 있는데도 눈에 잘 안띄고 간판도 촌스러워서(?) 손님이 많이 ...,"구역,눈,간판,지하,음료,데이트,홍초,카페,투어,카페,꼬메노,카페,호야,밑,쭉,눈,..."
78,"초코바나나 빽스치노에 샷 추가 해서 마셨어요. 예상되는 그 맛인데, 생각보다 많은 ...","초코바나나,빽,샷,예상,초코,칩,알알이,초코바나나,초코,아침,이용,더블,에스프레소,..."
262,림벅와플 덕수궁에 있는거 먹어보고 맛있엇던 기억에 cgv갔다가 샀다ㅋㅋ 한국에 아직...,"림벅,와플,덕수궁,기억,벨기에,리,쥬,와플,와플,플레인,메이플,와플,기선,수량,한정..."
170,조용한 분위기에서 정성스럽게 만든 한 잔을 마실 수 있음. 커피 맛있다. 자리가 아...,"정성,잔,커피,사운드,음악,잠시,정신,적당,아파트,촌,광장,면,개방,인스타,휴무,확..."


In [35]:
eat_review_by_user.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_review_by_user.csv')

In [ ]:
# pip install gensim

In [23]:
# w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
import gensim
w2v = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/ko/ko.bin')

In [24]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [25]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.wv.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


In [26]:
new_df

,침,가요,문후,관장,바이,방학,제발,그거,브랜드,명의,...,공략,국물,영문,리,번잡,묵,박스,내적,기,이사
0,0.711359,-1.707111,0.193378,-0.394486,-0.948398,-0.119451,-0.280704,0.163206,0.109528,1.784779,...,-2.108827,1.774642,1.013872,0.867246,-0.631302,2.325544,-0.378841,0.195684,-0.743848,0.483345
1,0.669836,-1.312540,-0.275783,1.357232,-1.939194,-0.083053,-0.124686,-0.038780,-1.144944,-1.036331,...,-0.535862,-0.819964,-1.365922,1.259495,0.050161,0.169798,-0.897125,-0.167671,1.885818,-0.687590
2,-0.090179,0.474495,0.225556,-0.807735,0.623011,0.754136,0.497279,0.156891,0.271200,-1.284180,...,-2.751044,0.512395,-0.641481,0.491653,0.131680,0.933448,0.304677,0.324100,-0.461646,2.053408
3,0.000663,-2.695328,0.014819,-0.908624,1.789394,-0.517791,-0.148131,-0.440327,1.582598,-1.500446,...,-1.300531,0.444904,1.586889,0.837418,-0.148977,0.281143,0.818394,0.187928,3.727956,-0.467874
4,1.168071,-0.486799,0.044058,0.535917,-0.075118,-0.081295,-0.626187,-0.385608,-1.016188,-2.278126,...,0.620326,0.818610,0.008602,-2.086288,-0.515272,-1.004438,-1.699146,0.423626,-4.793766,-1.421871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.303460,-0.098738,-0.088975,0.415246,-0.403882,1.283081,-0.047222,-0.108517,-0.807829,1.603693,...,-0.136348,0.802662,0.004296,-0.450704,0.099385,1.683796,-0.316072,-0.869979,0.631641,-0.357465
196,0.680116,-0.067639,-0.296710,-0.035135,1.516417,0.625747,-0.130246,0.285628,0.478126,-1.413939,...,0.747418,0.769095,-0.410953,0.686212,0.244273,-0.769025,0.480180,0.287093,1.726482,-1.061438
197,-0.068326,0.775789,-0.350417,-1.719429,-0.727421,-0.245953,-0.188887,0.026024,-0.807696,-1.868064,...,-0.144348,1.253738,0.061836,0.138566,0.043908,-1.403624,0.650051,1.152311,0.902872,0.262407
198,-0.392408,0.655866,0.196077,0.459362,0.271079,-0.230753,0.075873,-0.099165,2.269194,-0.252510,...,-0.642572,0.894820,0.206491,-1.934383,-0.012688,0.681759,-0.099252,1.010947,-1.686212,0.594321


In [27]:
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [28]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  35
1 :  39
2 :  44
3 :  97
4 :  3
5 :  61
6 :  5
7 :  56
8 :  1
9 :  39
10 :  1
11 :  11
12 :  53
13 :  68
14 :  1
15 :  5
16 :  36
17 :  39
18 :  10
19 :  2
20 :  19
21 :  16
22 :  5
23 :  4
24 :  3
25 :  2
26 :  14
27 :  3
28 :  16
29 :  25
30 :  54
31 :  30
32 :  27
33 :  1
34 :  20
35 :  3
36 :  10
37 :  16
38 :  6
39 :  1
40 :  507
41 :  2
42 :  24
43 :  56
44 :  1
45 :  1
46 :  223
47 :  14
48 :  38
49 :  43
50 :  1
51 :  100
52 :  1
53 :  21
54 :  1
55 :  73
56 :  7
57 :  181
58 :  5
59 :  2
60 :  64
61 :  1
62 :  1
63 :  1
64 :  7
65 :  50
66 :  10
67 :  24
68 :  1
69 :  19
70 :  32
71 :  1
72 :  4
73 :  34
74 :  2
75 :  1
76 :  33
77 :  1
78 :  1
79 :  4
80 :  7
81 :  16
82 :  110
83 :  86
84 :  1
85 :  44
86 :  1
87 :  25
88 :  50
89 :  23
90 :  3
91 :  10
92 :  11
93 :  1
94 :  40
95 :  1
96 :  25
97 :  5
98 :  35
99 :  66
100 :  2
101 :  1
102 :  12
103 :  51
104 :  30
105 :  24
106 :  1
107 :  2
108 :  1
109 :  15
110 :  23
111 :  19
112 :  1
113 :  12
114 :  3
115 : 

40

In [29]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [30]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,문후,향,모티브,뚜껑,연어,아르,그루,기호,마감,호선,기타,야마,왕,쓰레기,사,았,몬스터,즉시,브랜드,성질,려고,먼지,불만,예정,레,최,년,초록색,이용,컴퓨터,지점,침,뿌리,바이,낮,나머지,대학,출시,대한민국,스,정보,살,가능,바디,달라,재위,탄산,헝가리,기억,권력,지배,손질,합격,방학,상륙,롱,공부,미디엄,밀,오후,무한대,해지,여야,행위,독일어,신축,별개,관장,애,장식,사전,기,남쪽,명의,유래,관련,보충,대하,뿐,기반,입장,촬영,누,방,결혼,스크린,네,오프,안정,현금,다가,통일,다,늘,내지,속도,보이,찬성,폭,감성,전쟁,다그,가요,유원지,주류,의무,기념,신자,효과,동기,콜롬비아,동시,방향,의의,후반,구역,대회,도달,를,업무
1,제발,멍,네이버,쪽,야채,르,썩,문자,일로,0,신규,0,누나,칸막이,하리,0,포켓,곧,게임,값,려,입자,지적,추정,디,박,월,보라색,활용,전략,좌석,천사,턱,페,0,다섯,교수,판매,해외,머,0,거주,충분,하드,0,0,시금치,런던,진리,경고,0,지극,0,연한,0,트레이,고용,스트레이트,슬,오전,전이,0,0,0,한국어,부지,달리,초대,0,특색,혹시,0,동쪽,성인,발견,0,대체,0,0,바탕,의도,기재,콘,이대,0,옵션,0,트로피,소요,지불,여,통합,어,0,상,0,비롯,신청,편차,인지도,혁명,0,재즈,수운,교육,가정,0,교회,0,평가,차이나,현재,0,차이,초반,0,시국,정착,을,0
2,그거,골,동화,활,작물,피에르,장미,글자,계절,0,등,0,죽,휴식,도쿄,0,레벨,라며,스페셜,차원,0,온도,부담,예상,티,김,0,검은색,사용,작업,고가,신성,눈,스트,0,세부,대학교,수입,신라,캐,0,0,다양,사이드,0,0,장마,프랑스,지각,도전,0,가,0,독서,0,톰,퇴직,스터디,블,0,순환,0,0,0,한글,병원,일치,대사,0,특징,누구,0,면적,남자,0,0,수행,0,0,중심,태도,언급,족,순,0,버전,0,라운드,부작용,금액,면서,폐지,경우,0,피고,0,유지,부탁,몸무게,향기,0,0,노래,각지,문화,신분,0,0,0,과정,브라질,외,0,관계,초,0,축제,진출,0,0
3,적중,광저우,영화,윗,약품,0,가방,한자,겨울,0,종합,0,언니,고급,난,0,타입,일부러,보드,개념,0,함량,충격,추측,리,0,0,색상,0,기술,차선,단종,얼음,헤,0,일부,0,운영,외국,너,0,0,적절,톱,0,0,달구,미국,맥락,대치,0,본연,0,성균관,0,터스,졸업,카오스,울,0,하층,0,0,0,일본어,서초구,구별,주관,0,양식,보시,0,인구,자제,0,0,보완,0,0,배경,노력,참고,브루,대흥,0,머신,0,역전,치유,구입,0,분리,어서,0,만점,0,확보,의견,증가,시선,0,0,녹음,상권,역사,과실,0,0,0,연구,터키,시대,0,가치,0,0,전시회,건너,0,0
4,만족감,지리,이미지,상자,과일,0,모자,표기,크리스마스,0,개선,0,친척,컨테이너,나리,0,메탈,도중,아메리칸,공간,0,탄소,도움,예측,0,0,0,흰색,0,프로세스,사거리,악,이빨,쉬,0,각,0,제조,각국,트,0,0,애매,포크,0,0,유독,포르투,의식,권유,0,공존,0,경희대,0,릿,사정,보너스,델,0,유리,0,0,0,중국어,쇼핑몰,별도,축하,0,혼합,과연,0,0,간첩,0,0,판단,0,0,0,목적,찍,스트로,융,0,컬러,0,야구,집중,통화,0,편입,면,0,원내,0,발휘,요청,밝기,말투,0,0,멤버,늪,사상,원칙,0,0,0,과제,인도,과거,0,오류,0,0,행사,정복,0,0
5,사이다,주,이벤트,바닥,버섯,0,웃,0,일주일,0,특화,0,동생,포대,가미,0,0,뻔,샵,방법,0,공기,고민,0,0,0,0,띠,0,응용,고개,우상,모양,코,0,대부분,0,생산,우리나라,빙,0,0,확실,핀,0,0,비만,영국,자연,압박,0,하자,0,수업,0,어스,자격,훅,0,0,초고,0,0,0,영어,광화문,실제,역대,0,디자인,무슨,0,0,중학생,0,0,확장,0,0,0,시도,붙이,알바,주지,0,마이크로,0,챔피언,생기,고객,0,변경,며,0,불과,0,강조,0,최소,최대한,0,0,힙합,길목,회화,혜택,0,0,0,결과,동부,옛날,0,장애,0,0,세미나,출발,0,0
6,신랑,성,출판사,오른쪽,달걀,0,침대,0,날짜,0,지원,0,아버지,야외,짱,0,0,소식,레코드,집합,0,에너지,논란,0,0,0,0,빛깔,0,시스템,전면,이름,땅,폴,0,상위,0,제품,국내,0,0,0,완벽,퍼스트,0,0,막걸리,스페인,본질,집착,0,작정,0,전공,0,레이,근무,데스크,0,0,피드백,0,0,0,영문,본점,구분,등장,0,외형,얼마나,0,0,엄마,0,0,파악,0,0,0,계획,검색,솔,용인,0,서버,0,포지션,전복,예약,0,합치,더라도,0,동의,0,담당,0,퍼센트,흔적,0,0,팝,파도,친,생활,0,0,0,수준,케냐,당시,0,예외,0,0,단체,피신,0,0
7,빵집,상하이,로고,한쪽,제과,0,고양이,0,가을,0,표준화,0,아들,장갑,오기,0,0,얼마,로얄,모형,0,고체,영향,0,0,0,0,파란색,0,프로그램,차도,천국,조각,스파,0,분류,0,유통,대한,0,0,0,강하,루프,0,0,서빙,마니아,경험,무관심,0,세분,0,연희,0,크리스,0,최고급,0,0,곱,0,0,0,0,서울,흡사,올해,0,건축,상대방,0,0,어린이,0,0,이동,0,0,0,0,편집,마자,연,0,패스,0,상대,정체,비용,0,개정,듯이,0,폐기,0,역할,0,간격,평소,0,0,사운드,우림,경향,국적,0,0,0,분야,유엔,이외,0,문제,0,0,모임,격파,0,0
8,요가,불리,세대,글씨,기름,0,호랑이,0,일과,0,활성화,0,조카,수색,구로,0,0,잠깐,믹스,법칙,0,빛,불편,0,0,0,0,빨간색,0,환경,지하철,고하,바다,스포,0,각자,0,개발,전국,0,0,0,유일,포트,0,0,휴지,이탈리아,삶,부정,0,그대로,0,숙명,0,스파이,0,프롬,0,0,음향,0,0,0,0,공영,비슷,선정,0,구조,어디,0,0,연락,0,0,처리,0,0,0,0,작성,베르가,신한,0,시트,0,주심,비리,상품,0,개편,게,0,접수,0,수용,0,물가,감상,0,0,음악,겨울철,대중,제한,0,0,0,단계,국립공원,앞서,0,의미,0,

In [ ]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [36]:
del eat_info['Unnamed: 0']
eat_info.head()

KeyError: ignored

In [38]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_info_tags.csv')

In [39]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [40]:
res_matrix.head()

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
user_matrix.head()

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
416,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
252,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

  0%|          | 0/151 [00:00<?, ?it/s]

In [43]:
res_matrix.head()

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,76,33,15,11,150,0,8,0,17,0,14,0,1,12,22,0,1,11,9,4,0,6,8,1,50,0,0,1,3,2,5,18,7,13,1,11,0,1,9,2,1,0,3,27,2,0,133,13,17,6,0,12,0,3,0,6,1,40,1,0,8,0,0,0,1,25,3,1,1,1,16,1,1,7,1,0,17,0,0,0,2,2,25,17,0,3,0,5,4,14,1,1,9,0,5,0,7,0,3,14,0,0,3,17,4,4,0,0,0,3,0,7,0,3,0,0,0,1,0,0
1,7,4,0,1,15,0,2,0,4,0,0,0,0,1,3,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,7,0,1,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,4,0,1,0,1,1,2,0,0,0,0,3,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0
2,20,2,9,2,44,1,2,0,4,0,5,0,2,5,1,0,0,2,4,2,0,0,0,1,9,0,0,1,2,0,0,9,2,0,0,0,0,0,0,3,0,0,1,5,0,0,22,1,4,1,0,5,0,1,0,5,0,9,0,1,1,0,0,0,0,11,0,1,0,2,3,1,0,4,0,0,3,0,0,0,1,2,8,8,0,1,0,3,1,0,0,0,0,0,4,0,3,0,3,5,0,0,0,0,0,1,0,0,0,1,1,2,0,1,0,0,0,1,1,0
3,19,1,2,1,38,0,0,0,2,0,3,0,0,3,1,0,0,1,2,5,0,0,3,1,0,11,0,0,0,0,2,1,6,0,0,0,0,0,1,0,0,0,2,5,1,0,7,0,1,0,0,1,0,0,0,0,1,3,0,0,2,0,0,0,0,4,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,0,2,2,0,1,0,0,0,3,0,0,0,1,1,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0
4,31,3,83,6,71,0,2,1,5,0,2,0,0,5,5,0,1,6,6,15,0,0,1,0,11,0,0,6,0,3,2,4,6,1,1,6,1,4,0,2,0,0,0,6,0,0,17,0,8,4,0,6,0,2,0,5,0,31,0,0,2,0,0,1,0,15,1,5,0,10,2,0,0,1,1,1,10,0,0,1,0,1,7,15,0,6,0,1,4,2,0,6,4,0,0,0,6,0,2,14,0,0,2,3,10,0,0,0,0,4,0,3,0,1,0,0,0,0,1,0


In [44]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]


  0%|          | 0/600 [00:00<?, ?it/s]

In [45]:
user_matrix.head()

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
416,2,0,1,1,2,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
252,57,13,36,17,107,0,10,0,9,0,6,0,3,10,16,0,0,4,6,5,0,1,5,4,5,0,0,0,0,2,4,12,19,2,0,2,0,5,0,1,0,0,2,9,1,0,68,0,7,0,0,15,0,0,0,15,1,25,1,0,4,0,0,0,0,13,1,0,0,2,4,0,0,6,1,0,3,0,0,0,0,3,13,21,0,7,0,1,8,3,0,0,1,0,1,0,4,0,3,7,0,0,1,10,3,3,0,0,0,1,0,1,0,4,0,0,2,0,1,0
148,29,0,3,0,109,0,5,0,9,0,2,0,1,2,6,0,0,7,4,0,0,0,1,1,17,0,1,0,0,0,3,3,6,6,1,1,0,0,0,1,0,0,0,6,0,0,34,0,4,4,0,3,0,2,0,12,0,22,0,2,7,0,0,0,0,3,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,9,4,0,10,0,0,0,1,0,6,1,0,1,0,2,0,1,2,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
266,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
406,0,0,2,0,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
user_matrix.reset_index()

Output hidden; open in https://colab.research.google.com to view.

In [47]:
user_matrix.rename(columns ={'index':'u_id'})

Output hidden; open in https://colab.research.google.com to view.

In [48]:
user_matrix.head()

,group_40,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
416,2,0,1,1,2,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
252,57,13,36,17,107,0,10,0,9,0,6,0,3,10,16,0,0,4,6,5,0,1,5,4,5,0,0,0,0,2,4,12,19,2,0,2,0,5,0,1,0,0,2,9,1,0,68,0,7,0,0,15,0,0,0,15,1,25,1,0,4,0,0,0,0,13,1,0,0,2,4,0,0,6,1,0,3,0,0,0,0,3,13,21,0,7,0,1,8,3,0,0,1,0,1,0,4,0,3,7,0,0,1,10,3,3,0,0,0,1,0,1,0,4,0,0,2,0,1,0
148,29,0,3,0,109,0,5,0,9,0,2,0,1,2,6,0,0,7,4,0,0,0,1,1,17,0,1,0,0,0,3,3,6,6,1,1,0,0,0,1,0,0,0,6,0,0,34,0,4,4,0,3,0,2,0,12,0,22,0,2,7,0,0,0,0,3,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,9,4,0,10,0,0,0,1,0,6,1,0,1,0,2,0,1,2,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
266,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
406,0,0,2,0,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
user_matrix.set_index('u_id', drop=True, inplace=True)


KeyError: ignored

In [51]:
km_df.count()

group_40     507
group_0       35
group_1       39
group_2       44
group_3       97
group_4        3
group_5       61
group_6        5
group_7       56
group_8        1
group_9       39
group_10       1
group_11      11
group_12      53
group_13      68
group_14       1
group_15       5
group_16      36
group_17      39
group_18      10
group_19       2
group_20      19
group_21      16
group_22       5
group_23       4
group_24       3
group_25       2
group_26      14
group_27       3
group_28      16
group_29      25
group_30      54
group_31      30
group_32      27
group_33       1
group_34      20
group_35       3
group_36      10
group_37      16
group_38       6
group_39       1
group_41       2
group_42      24
group_43      56
group_44       1
group_45       1
group_46     223
group_47      14
group_48      38
group_49      43
group_50       1
group_51     100
group_52       1
group_53      21
group_54       1
group_55      73
group_56       7
group_57     181
group_58      

In [52]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

,u_id,p_id,rating
0,0,15,3
1,1,63,3
2,1,63,3
3,1,57,5
4,1,63,3


In [59]:
res_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_res_matrix.csv')
user_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_user_matrix.csv')
eat_review_score.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_score_board.csv',index=False)
km_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_eat_cluster.csv',index=False)
km_df.count().to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_group_count.csv')